# MCT4052 Workshop 4b: Support Vector Machine Regressor

*Author: Stefano Fasciani, stefano.fasciani@imv.uio.no, Department of Musicology, University of Oslo.*

This notebook includes the same regression problem of Workshop 4a, but in this case we use a more flexible (and likely powerful) [Support Vector Regression](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html) regression technique, which is based on Support Vector Machines (SVM).

Usually SVM are used for classification stasks, this [post](https://towardsdatascience.com/an-introduction-to-support-vector-regression-svr-a3ebc1672c2) explains how they can be used for regression. You should also read [section 1.4.2 of the scikitlearn SVM page](http://scikit-learn.org/stable/modules/svm.html).

In [1]:
import numpy as np
import pandas as pd
import librosa, librosa.display
import matplotlib.pyplot as plt
import matplotlib.style as ms
#ms.use("seaborn-v0_8") 
import IPython.display as Ipd
import os
import sklearn

%matplotlib inline
%config IPCompleter.greedy=True

In [2]:
sr = 22050

#Instead of writing the code to extract the features and target value we define a function,
#which is more elegant, it's reusable (shorter code) and makes the following code more readable.
#For practicality, the vector features are flattenedm so that that can be stored on a row of a N-dim array

#Mind that the file name is not important here because there is no label to consider (as in the previous examples)

def extract_features_targets(filename, sr=sr):
    
    signal, dummy = librosa.load(filename, sr=sr, mono=True)
    
    features = librosa.feature.melspectrogram(y=signal, sr=sr)
    features = features.flatten()
    
    target = np.mean(librosa.feature.rms(y=signal))
    
    return features, target


#creating an array of zeros of the proper size where we will store computed features and lables
filenames = os.listdir('./data/examples3')

#to set the right number of columns, we call the extract_features_target() function and get the size of the features
num_of_features = extract_features_targets('./data/examples3/'+filenames[0], sr=sr)[0].size
num_of_targets = extract_features_targets('./data/examples3/'+filenames[0], sr=sr)[1].size
features = np.zeros((len(filenames),num_of_features)) 
targets = np.zeros((len(filenames),num_of_targets))

for i in range(len(filenames)):
    #print('processing',filenames[i])
    features[i,:], targets[i,:] = extract_features_targets('./data/examples3/'+filenames[i], sr)

print('Done!')

Done!


In [3]:
#this step converts the targets from a numpy array to 
#a Pandas series, which allows backtracing poorly performing examples
#scikitlearn can handle features or targets also as Pandas formats
#mind that this wont work with multiple target values (as the data must be dimensionless)
targets = pd.Series(targets.flatten())

In [4]:
from sklearn.model_selection import train_test_split

#splitting the dataset in training and testing parts
feat_train, feat_test, tar_train, tar_test = train_test_split(features, targets, test_size=0.3, random_state=5)

### Creating, training and testing the regressor

In [5]:
from sklearn.svm import SVR

# Create SVR regression object
regr = SVR(C=1.0, epsilon=0.001, kernel='rbf')

# Train the model using the training sets
regr.fit(feat_train, tar_train)

# Make predictions using the testing set
tar_predict = regr.predict(feat_test)

In [6]:
#computing a set of performance metrics

#mean squared error (lower the better)
print('Mean squared error: %.4f'% sklearn.metrics.mean_squared_error(tar_test, tar_predict))

#mean absolute error (lower the better)
print('Mean absolute error: %.4f'% sklearn.metrics.mean_absolute_error(tar_test, tar_predict))

#maximum error (lower the better)
print('Max error squared error: %.4f'% sklearn.metrics.max_error(tar_test, tar_predict))

#median absolute error (lower the better)
print('Median absolute error: %.4f'% sklearn.metrics.median_absolute_error(tar_test, tar_predict))

#coefficient of determination (r2 score): 1 is perfect prediction (it can get arbitrary negative)
print('Coefficient of determination (R2 score): %.4f'% sklearn.metrics.r2_score(tar_test, tar_predict))

#explained variance score: 1 is perfect prediction (it can get arbitrary worse)
print('Explained variance score: %.4f'% sklearn.metrics.explained_variance_score(tar_test, tar_predict))


Mean squared error: 0.0006
Mean absolute error: 0.0172
Max error squared error: 0.0653
Median absolute error: 0.0105
Coefficient of determination (R2 score): 0.8763
Explained variance score: 0.8845


### Displaying errors and files with error above a given treshold

In [7]:
#displaying the individual errors
errors = np.absolute(tar_test - tar_predict)
print(errors)

#displaying names of poorly predicted files
error_threshold = 0.01

print('test examples above the error_threshold')
for index, value in errors.items():
    if value > error_threshold: #setting an arbitrary error threshold
        print(filenames[index], value)


40    0.002810
37    0.015772
57    0.008701
60    0.033411
77    0.007677
20    0.010369
10    0.010529
48    0.000822
34    0.015369
28    0.001484
49    0.037888
32    0.001150
17    0.032080
22    0.030327
23    0.012609
24    0.001281
12    0.003423
63    0.009125
85    0.036609
43    0.001306
35    0.039637
13    0.004674
54    0.010780
74    0.034132
55    0.001342
89    0.065312
6     0.034946
dtype: float64
test examples above the error_threshold
mallet_acoustic_062-096-050.wav 0.015771913605386406
reed_acoustic_011-069-100.wav 0.03341114839659473
flute_synthetic_000-080-050.wav 0.010369303842028554
string_acoustic_080-049-127.wav 0.010528865958622263
mallet_acoustic_062-093-050.wav 0.015369153580641781
reed_acoustic_011-059-025.wav 0.03788759560703736
organ_electronic_057-079-075.wav 0.03207958823175899
string_acoustic_080-047-100.wav 0.030327469766432497
mallet_acoustic_062-102-075.wav 0.01260923826187102
reed_acoustic_011-073-075.wav 0.03660920127417855
reed_acoustic_011-04

### Follow-up activity

1. Identify the cells (as well as the lines of code within the cells) of this notebook that are absolutely necessary to train and test the regressor.
2. Make a fair assessment of the error, explore the dymanic range/magnitude of the target values versus the magnitude of the errors (and the performance metrics).
3. Try to use the [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) to scale the features, or also the target values, assess the change in the performances. If you use the scaler on the target value, apply the inverse_transform() method of the scaler to convert the regression output to the actual traget values.
4. Explore the same technique to predict different features used as input and/or output fo the regressor. Or experiment by changing the parameters used to compute the features, such as (sampling rate, the n_fft, hop_lenght, n_mels, etc.)
5. Experiment by changing the hyperparameters of the regressor.